If users already have de-identification datasets in BIO format, users can directly conduct PHI Augmentation by modifying the following code.

Please check whether the PHI types in your own dataset are included in the provided named-entity lists in 'all_PHI_from_internet' directory.

In [ ]:
import random

file_path_entity = 'C:/Users/zhou/Desktop/all_PHI_from_internet/'
labels_in_2006 = ['PATIENT', 'DOCTOR', 'HOSPITAL', 'IDNUM', 'DATE', 'PHONE', 'AGE', 'ZIP',
                  'MEDICALRECORD', 'ORGANIZATION', 'PROFESSION', 'USERNAME', 'LOCATION']

label_content = [[] for i in range(len(labels_in_2006))]
for k in range(len(labels_in_2006)):
    file_path = file_path_entity + labels_in_2006[k] + '.txt'
    with open(file_path, 'r', encoding='utf-8') as f_w:
        content = f_w.readlines()
        for j in content:
            label_content[k].append(j.strip())


def replace_PHI(label, line_number):
    preserve_list = []
    if label in labels_in_2006:
        label_index = labels_in_2006.index(label)
        words_list = label_content[label_index]

        random.seed(global_seed)  # every epoch, it produces different dataset, also reimplementable.
        random.shuffle(words_list)
        words_num = len(words_list)

        random.seed(line_number)  # line_number is defined for specific PHI instance
        random_index = random.randint(0, words_num - 1)
        random_w = words_list[random_index].strip()  # words_list is different at different epoch

        words_num = random_w.split()
        if len(words_num) == 1:
            new_line = random_w+'   '+'B-'+label
            preserve_list.append(new_line)
        if len(words_num) > 1:
            for i in range(len(words_num)):
                if i == 0:
                    new_line = words_num[i]+'   '+'B-'+label
                    preserve_list.append(new_line)
                if i > 0:
                    new_line = words_num[i]+'   '+'I-'+label
                    preserve_list.append(new_line)
    return preserve_list


file_path_2014 = 'C:/Users/zhou/Desktop/processed_2014_train.txt'
global_seed = 2
repalced_file_path = 'C:/Users/zhou/Desktop/new_replaced_2014_train_seed'+str(global_seed)+'.txt'


reserve_content = []
with open(file_path_2014, 'r', encoding='utf-8') as f_wr:
    content = f_wr.readlines()
    for row in range(0, len(content)-1):
        if len(content[row]) == 1:  # cope with empty line
            reserve_content.append('')
        if len(content[row]) != 1 and len(content[row+1]) != 1:
            all_token_Next = content[row + 1].split()
            all_token = content[row].split()
            this_line_label = all_token[-1]
            next_line_label = all_token_Next[-1]
            if this_line_label.startswith('O'):
                new_line = all_token[0]+'   '+all_token[-1]
                reserve_content.append(new_line)
                continue
            if this_line_label.startswith('B-') and next_line_label.startswith('I-'):
                continue
            if this_line_label.startswith('B-') and not next_line_label.startswith('I-'):
                label = this_line_label.split('-')[-1]
                new_words = replace_PHI(label, row)
                for k in new_words:
                    reserve_content.append(k)
            if this_line_label.startswith('I-') and next_line_label.startswith('I-'):
                continue
            if this_line_label.startswith('I-') and not next_line_label.startswith('I-'):
                label = this_line_label.split('-')[-1]
                # print(label)
                new_words = replace_PHI(label, row)
                # print(new_words)
                for k in new_words:
                    reserve_content.append(k)

        if len(content[row]) != 1 and len(content[row+1]) == 1:
            all_token = content[row].split()
            this_line_label = all_token[-1]
            if this_line_label.startswith('B-') or this_line_label.startswith('I-'):
                label = this_line_label.split('-')[-1]
                new_words = replace_PHI(label, row)
                for k in new_words:
                    reserve_content.append(k)
            if this_line_label.startswith('O'):
                new_line = all_token[0]+'   '+all_token[-1]
                reserve_content.append(new_line)
                continue


with open(repalced_file_path, 'w', encoding='utf-8') as f_w:
    for line in reserve_content:
        f_w.writelines(line+'\n')